In [ ]:
import utils, shapely_utils as shu, python_utils as pu
import bezier
import numpy as  np
from shapely import LineString, MultiLineString, GeometryCollection
import shapely as sh

In [7]:
# imports
from pathlib import Path, PurePath
import sys

In [8]:

def bball_net(radius,m=0.15,n=0.25,total_slices=12,len_straight=4, prop_h=0.5):
    curve = bezier.Curve(np.asfortranarray(np.matrix([[0,0],[n/4,m],[n,prop_h*m]]).transpose()), degree=2)
    line = LineString([[-n/3,-n/len_straight],[0,0]])
    # return line
    l = sh.line_merge(MultiLineString([line,shu.curve_to_ls(curve)]))
    # return line
    a,b,c,d = l.bounds
    x_dist = c-a
    y_dist = d-b
    w_line,h_line = shu.size(line)
    
    p = sh.line_merge(MultiLineString([l,sh.affinity.translate(sh.affinity.scale(l, -1, 1),x_dist)]))
    # return l
    # return p
    w,h = shu.size(p)
    scale_prop = radius/w
    p = sh.affinity.scale(p, scale_prop, scale_prop)
    p2 = sh.affinity.translate(sh.affinity.scale(p, 1, -1),0,- h * scale_prop + h_line*scale_prop*2)
    lip = GeometryCollection([p,p2])
    # return lip
    # return lip
    # return lip
    tots_lips = []
    mergers_entre_lips = []
    def getLines(last_line, new_line):
        line = LineString([list(list(new_line.geoms)[1].coords)[0],list(list(last_line.geoms)[0].coords)[0]])
        line2 = LineString([list(list(new_line.geoms)[0].coords)[-1],list(list(last_line.geoms)[1].coords)[-1]])
        return [line, line2]
    first_line = None
    for i in range(int(total_slices/2)):
        new_line = sh.affinity.rotate(lip, i*(360/total_slices))
        if (i>0):
            mergers_entre_lips += getLines(last_line, new_line)
        else:
            first_line = new_line
        tots_lips.append(new_line)
        last_line = new_line
    # inverse
    line = LineString([list(list(first_line.geoms)[1].coords)[0],list(list(last_line.geoms)[1].coords)[-1]])
    line2 = LineString([list(list(first_line.geoms)[0].coords)[-1],list(list(last_line.geoms)[0].coords)[0]])
    mergers_entre_lips += [line, line2]
    # return tots_lips
    # print(len(tots_lips))
    # return [tots_lips,mergers_entre_lips]
    return shu.StructuredCollection(lips=tots_lips, lips_mergers=mergers_entre_lips)

In [ ]:
pu.reload(shu)

In [18]:
pu.reload(shu)

In [15]:
upB = bball_net(50, 0.15,0.25,12,3.7,0.85)
downB = bball_net(50, 0.15,0.25,12,3.7,0.85).translate(0,-100)
# downB2 = bball_net(50, 0.15,0.25,12,3.7,0.85).translate(0,90)
# shu.print([upB.geo, downB.geo])

In [ ]:
top_mergers = 6
top_mergers_up = shu.reorder_geos(upB['lips_mergers'], "S", 6)
top_mergers_down = shu.reorder_geos(downB['lips_mergers'], "N", 6)
springs_top = []
springs_bottom = []
for i in range(top_mergers):
    l1 = top_mergers_up.geoms[i]
    spring_l1 = shu.spring(l1,3,1,0.8)
    springs_top.append(spring_l1)
    l2 = top_mergers_down.geoms[i]
    spring_l2 = shu.spring(l2,3,1,0.8)
    springs_bottom.append(spring_l2)

In [10]:
# shu.print([reorder_geos(upB['lips_mergers'], "E", 6),top_mergers_down])

In [11]:
# i = 0
# flow_total = []
# while (len(springs_bottom)>0 or len(springs_top)>0):
#     list_to_pick = springs_top
#     if i%2==1:
#         list_to_pick = springs_bottom
#     item = list_to_pick.pop()
#     flow_total.append(item)
#     i+=1

In [20]:
# last = None
# first_item = None
# guides = []
# for i,springs in enumerate(shu.iter_two(flow_total)):
#     s1, s2 = springs
#     if (not last is None):
#         guides.append(LineString([last, shu.first(s1)]))
#     guides.append(LineString([shu.last(s1), shu.first(s2)]))
#     last = shu.last(s2)
# last = None
first_item = None
guides = []
reorder_g = shu.reorder_geos(springs_top, "E")
for i,springs in enumerate(zip(shu.reorder_geos(springs_top, "E"), shu.reorder_geos(springs_bottom, "E"))):
    s1, s2 = springs
    # if (not last is None):
    guides.append(LineString([shu.first(s1), shu.first(s2)]))
    guides.append(LineString([shu.last(s1), shu.last(s2)]))
    # last = shu.last(s2)

In [ ]:
sh.affinity.scale(shu.print(guides),50,50)

In [ ]:
shu.print(guides)

In [ ]:
spring_g = shu.spring(l,3,1,0.8)
spring_g

In [ ]:
springs_lip_merger = GeometryCollection([shu.spring(l,3,None,0.6) for l in shu.geoms(upB['lips_mergers'])])
# sh.affinity.scale(shu.print([springs_lip_merger, upB['lips_mergers']]),500,500)
# springs_lip_merger

In [24]:
pu.reload(shu)

In [ ]:
# line = LineString([(0,1),(9,3)])
# shu.spring(line, 3,None,0.8)
# shu.print([shu.spring(line, 3, None,0.8)])

In [27]:
# x_off,y_off = line.bounds


In [53]:
# num_loops = 9
# line = LineString([(0,0),(25,0)])
# radius = line.length/(num_loops+2)
# # radius = 0.25*num_loops
# all_circles = []
# for i in range(num_loops):
#     circle = shu.Circle((radius*(i+1),0), radius)
#     all_circles.append(circle)
# springs = []
# for c1, c2 in shu.iter_two(all_circles):
#     springs.append(createSpring(c1, c2))
pu.reload(shu)

In [ ]:
shu.print([g.centroid for g in shu.coords(guides[1].segmentize(5))])

In [ ]:
# guides

In [61]:
pu.reload(shu)

In [ ]:
# print(p_noise.one(2))
segments = [LineString(l) for l in shu.iter_two(shu.coords(guides[1].segmentize(10)))]
import random
random.seed(10)
curves = []
for s in segments:
    norm = shu.sum_point_vector(s.centroid,shu.normal_vector(s), random.random()*10-5)
    c = shu.last(norm)
    curv = shu.curved_path(shu.first(s),shu.last(s), c,0)
    curves.append(curv)


In [ ]:
curves

In [ ]:
sh.line_merge(curves)

In [ ]:
shu.print([segments[0], ])

In [ ]:
print(p.one(0))

In [121]:
def wiggleLine(line):
    line

In [ ]:
from shapely.geometry import Point
import matplotlib.pyplot as plt

a = (0, 0)
b = (10, 0)
c = (9.5, 5)

line = shu.curved_path(a, b, c, curvature=1)
shu.print([line,Point(a),Point(b),Point(c)])

In [ ]:
shu.join(upB,sh.affinity.translate(upB, 0, d*1.5))

In [ ]:
GeometryCollection([shu.Cirºcle(upB.centroid, d/2)]+list(upB_half.geoms))

In [ ]:

square = sh.affinity.translate(shu.Rect(upB.centroid,d/2,d/2).union(shu.Rect(upB.centroid,-d/2,d/2)),0,-d/2)
upB_half = shu.intersection_collection(upB, square)
downB = upB.copy()
square2 = sh.affinity.translate(shu.Rect(downB.centroid,d/2,d/2).union(shu.Rect(downB.centroid,-d/2,d/2)),0,0)
downB_half = shu.intersection_collection(downB, square2)
# crec k vull fer que totes les geometries .rotate .translate .affine_transform existeixin
bottom = sh.affinity.translate(downB_half, 0, -shu.size(downB_half)[1]*3)
GeometryCollection([bottom, upB_half])
all_lines = []
for i,ls in enumerate(bottom.geoms):
    print(upB_half.geoms)
    # print(upB_half.geoms[i].coords[0],ls[0])

In [16]:
v = LineString([[1,0],[1,2]])
hs = [{"line": LineString([[0,(0.5+i)*v.length/5],[2,(0.5+i)*v.length/5]]), "orientation": ((i%2==0)*2)-1} for i in range(5)]

In [ ]:
for h in hs:
    intersection = h['line'].intersection(v)
    h['intersection'] = intersection
hs.sort(key=lambda a: sh.distance(a['intersection'],v.boundary.geoms[0]))
hs

In [ ]:

import numpy as np
current_inline = v.boundary.geoms[0]
curves = []
intensity = 0.5
for i,h in enumerate(hs):
    if (len(hs)>i+1):
        print(h['intersection'],hs[i+1]['intersection'])
        next_inline = LineString([h['intersection'],hs[i+1]['intersection']]).centroid
    else:
        next_inline = v.boundary.geoms[1]
    line = h['line']
    point = h['intersection']

    middle_node = shu.move_point_along_line(point, line, intensity*h['orientation'])
    
    nodes = [list(a) for a in [current_inline.coords[0], middle_node.coords[0], next_inline.coords[0]]]
    # print(nodes)
    # sino transposes es pensa que ed 3d bezier
    curve = bezier.Curve.from_nodes(np.asfortranarray(np.transpose(nodes)))
    # enllaçar curves?
    curves.append(curve)
    current_inline = next_inline

In [ ]:
MultiLineString([shu.curve_to_ls(c) for c in curves]+[v]+[h['line'] for h in hs])